In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xarray as xr

In [4]:
# Load the NetCDF file
file_path = 'adt.nc'
data = xr.open_dataset(file_path)

# Inspect the contents
data

<xarray.Dataset> Size: 289MB
Dimensions:    (time: 1254, latitude: 120, longitude: 240)
Coordinates:
  * latitude   (latitude) float32 480B 0.125 0.375 0.625 ... 29.38 29.62 29.88
  * longitude  (longitude) float32 960B 40.12 40.38 40.62 ... 99.38 99.62 99.88
  * time       (time) datetime64[ns] 10kB 2020-01-01 2020-01-02 ... 2023-06-07
Data variables:
    adt        (time, latitude, longitude) float64 289MB ...
Attributes: (12/45)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         http://climate.copernicus.eu/c3s-user-se...
    creator_email:                   http://climate.copernicus.eu/c3s-user-se...
    ...                              ...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-01-01T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-12-31T12:00:00Z
    title:                           DT merged two satellites Global Ocean Gr...
    copernicusmarine_version:        1.3.3

In [5]:
adt_data = data.adt.values

In [6]:
adt_data.shape

(1254, 120, 240)

In [7]:
adt_tensor = torch.from_numpy(adt_data)

In [8]:
adt_tensor = adt_tensor.reshape((1254,28800))
adt_tensor.shape

torch.Size([1254, 28800])

In [9]:
adt_tensor = torch.transpose(adt_tensor, 1,0)
adt_tensor.shape

torch.Size([28800, 1254])

In [10]:
adt_tensor.nan_to_num_(adt_tensor.nanmean())

tensor([[0.8774, 0.8774, 0.8774,  ..., 0.8774, 0.8774, 0.8774],
        [0.8774, 0.8774, 0.8774,  ..., 0.8774, 0.8774, 0.8774],
        [0.8774, 0.8774, 0.8774,  ..., 0.8774, 0.8774, 0.8774],
        ...,
        [0.8774, 0.8774, 0.8774,  ..., 0.8774, 0.8774, 0.8774],
        [0.8774, 0.8774, 0.8774,  ..., 0.8774, 0.8774, 0.8774],
        [0.8774, 0.8774, 0.8774,  ..., 0.8774, 0.8774, 0.8774]],
       dtype=torch.float64)

In [11]:
adt_tensor.isnan().sum()

tensor(0)

In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from torch import nn
from torch.utils.data import DataLoader, Dataset,TensorDataset

In [13]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(adt_tensor)

In [14]:
dataset

array([[0.44373553, 0.43918507, 0.43298304, ..., 0.41418864, 0.41333715,
        0.41304395],
       [0.44373553, 0.43918507, 0.43298304, ..., 0.41418864, 0.41333715,
        0.41304395],
       [0.44373553, 0.43918507, 0.43298304, ..., 0.41418864, 0.41333715,
        0.41304395],
       ...,
       [0.44373553, 0.43918507, 0.43298304, ..., 0.41418864, 0.41333715,
        0.41304395],
       [0.44373553, 0.43918507, 0.43298304, ..., 0.41418864, 0.41333715,
        0.41304395],
       [0.44373553, 0.43918507, 0.43298304, ..., 0.41418864, 0.41333715,
        0.41304395]])

In [15]:
train_size = int(dataset.shape[1]*0.7)
test_size = int(dataset.shape[1]*0.2)
evalute_size = dataset.shape[1]-train_size-test_size
train_size,test_size,evalute_size

(877, 250, 127)

In [16]:
train,test,evaluate = dataset[:,0:train_size], dataset[:,train_size:test_size+train_size], dataset[:,test_size+train_size:len(dataset)]

In [17]:
train.shape,test.shape,evaluate.shape

((28800, 877), (28800, 250), (28800, 127))

In [18]:
def to_sequences(dataset,seq_size=1):
    x = []
    y = []
    for i in range(dataset.shape[1]-seq_size):
        window = dataset[:,i:(i+seq_size)]
        x.append(window)
        y.append(dataset[:,i+seq_size])
    x,y = np.array(x), np.array(y)
    return torch.tensor(x,dtype=torch.float32), torch.tensor(y,dtype=torch.float32)

In [19]:
seq_size = 10

In [20]:
trainX, trainy = to_sequences(train,seq_size)
testX, testy = to_sequences(test,seq_size)
evaluateX, evaluatey = to_sequences(evaluate,seq_size)

In [21]:
trainy = trainy.unsqueeze(-1)
testy = testy.unsqueeze(-1)
evaluatey = evaluatey.unsqueeze(-1)

In [22]:
print(trainX.shape, trainy.shape)
print(testX.shape, testy.shape)

torch.Size([867, 28800, 10]) torch.Size([867, 28800, 1])
torch.Size([240, 28800, 10]) torch.Size([240, 28800, 1])


In [23]:
class CustomDataset(Dataset):
    def __init__(self,x_tensor,y_tensor):
        self.xdomain = x_tensor
        self.ydomain = y_tensor
    def __len__(self):
        return len(self.xdomain)
    def __getitem__(self,index):
        return (self.xdomain[index],self.ydomain[index])

In [25]:
train_dataset = CustomDataset(trainX,trainy)
test_dataset = CustomDataset(testX,testy)
evaluate_dataset = CustomDataset(evaluateX,evaluatey)

In [26]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
evaluate_dataloader = DataLoader(evaluate_dataset, batch_size=1, shuffle=False)

In [27]:
class ADTForecaster(nn.Module):
    def __init__(self, input_size=10, hidden_size=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 28800)  # Directly output all locations
        )

    def forward(self, x):
        out, _ = self.lstm(x)  # (batch, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])  # Use last timestep's output
        return out.unsqueeze(-1)  # Add output dimension (batch, 28800, 1)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ADTForecaster().to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [29]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

In [31]:
accumulation_steps = 8

# Training Loop
def train_model(model, train_loader, val_loader, epochs=50):
    best_val_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        optimizer.zero_grad()

        for i, (batch_x, batch_y) in enumerate(train_loader):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            outputs = model(batch_x)
            loss = criterion(outputs, batch_y) / accumulation_steps
            loss.backward()

            if (i + 1) % (accumulation_steps *10) == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                optimizer.zero_grad()

            train_loss += loss.item() * batch_x.size(0)

        # Validation
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                val_loss += criterion(outputs, batch_y).item() * batch_x.size(0)

        # Update learning rate
        scheduler.step(val_loss)

        # Print progress
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)
        print(f'Epoch {epoch+1:02} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            patience = 0
        else:
            patience += 1
            if patience >= 5:
                print("Early stopping")
                break
train_model(model, train_dataloader, test_dataloader, epochs=50)

Epoch 01 | Train Loss: 0.0243 | Val Loss: 0.1489
Epoch 02 | Train Loss: 0.0160 | Val Loss: 0.0683
Epoch 03 | Train Loss: 0.0054 | Val Loss: 0.0328
Epoch 04 | Train Loss: 0.0024 | Val Loss: 0.0067
Epoch 05 | Train Loss: 0.0013 | Val Loss: 0.0057
Epoch 06 | Train Loss: 0.0011 | Val Loss: 0.0055
Epoch 07 | Train Loss: 0.0010 | Val Loss: 0.0051
Epoch 08 | Train Loss: 0.0010 | Val Loss: 0.0055
Epoch 09 | Train Loss: 0.0010 | Val Loss: 0.0051
Epoch 10 | Train Loss: 0.0010 | Val Loss: 0.0053
Epoch 11 | Train Loss: 0.0010 | Val Loss: 0.0051
Epoch 12 | Train Loss: 0.0009 | Val Loss: 0.0052
Early stopping


# Evalutation with 10%

In [35]:
def evaluate_model(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            outputs = outputs.squeeze(-1).cpu().numpy()
            batch_y = batch_y.squeeze(-1).cpu().numpy()

            outputs = outputs.T
            batch_y = batch_y.T

            outputs_denorm = scaler.inverse_transform(outputs[:1254, :].T)
            targets_denorm = scaler.inverse_transform(batch_y[:1254, :].T)

            predictions.append(outputs_denorm)
            actuals.append(targets_denorm)

    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)

    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)

    print(f'Test RMSE: {rmse:.4f}')
    print(f'Test MAE: {mae:.4f}')

evaluate_model(model, evaluate_dataloader, scaler)


Test RMSE: 0.0717
Test MAE: 0.0563


In [36]:
def evaluate_accuracy(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            outputs = outputs.squeeze(-1).cpu().numpy()
            batch_y = batch_y.squeeze(-1).cpu().numpy()

            outputs_fixed = outputs[:, :1254]
            batch_y_fixed = batch_y[:, :1254]

            outputs_denorm = scaler.inverse_transform(outputs_fixed)
            targets_denorm = scaler.inverse_transform(batch_y_fixed)

            predictions.append(outputs_denorm)
            actuals.append(targets_denorm)

    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)

    mae = mean_absolute_error(actuals, predictions)

    mean_actual = np.mean(np.abs(actuals))

    # Compute Accuracy
    accuracy = max(0, 100 - (mae / mean_actual * 100))

    print(f"Test Accuracy: {accuracy:.2f}%")

evaluate_accuracy(model, evaluate_dataloader, scaler)

Test Accuracy: 93.46%


Testing data Evaluation

In [32]:
def evaluate_model(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            outputs = outputs.squeeze(-1).cpu().numpy()
            batch_y = batch_y.squeeze(-1).cpu().numpy()

            outputs = outputs.T
            batch_y = batch_y.T

            outputs_denorm = scaler.inverse_transform(outputs[:1254, :].T)
            targets_denorm = scaler.inverse_transform(batch_y[:1254, :].T)

            predictions.append(outputs_denorm)
            actuals.append(targets_denorm)

    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)

    # Compute evaluation metrics
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)

    print(f'Test RMSE: {rmse:.4f}')
    print(f'Test MAE: {mae:.4f}')

evaluate_model(model, test_dataloader, scaler)


Test RMSE: 0.0660
Test MAE: 0.0521


In [33]:
def evaluate_accuracy(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            outputs = outputs.squeeze(-1).cpu().numpy()
            batch_y = batch_y.squeeze(-1).cpu().numpy()

            outputs_fixed = outputs[:, :1254]
            batch_y_fixed = batch_y[:, :1254]

            outputs_denorm = scaler.inverse_transform(outputs_fixed)
            targets_denorm = scaler.inverse_transform(batch_y_fixed)

            predictions.append(outputs_denorm)
            actuals.append(targets_denorm)

    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)

    mae = mean_absolute_error(actuals, predictions)

    mean_actual = np.mean(np.abs(actuals))

    accuracy = max(0, 100 - (mae / mean_actual * 100))

    print(f"Test Accuracy: {accuracy:.2f}%")

evaluate_accuracy(model, test_dataloader, scaler)

Test Accuracy: 93.94%


Checking for overfitting

In [34]:
def evaluate_accuracy(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            outputs = outputs.squeeze(-1).cpu().numpy()
            batch_y = batch_y.squeeze(-1).cpu().numpy()

            outputs_fixed = outputs[:, :1254]
            batch_y_fixed = batch_y[:, :1254]

            outputs_denorm = scaler.inverse_transform(outputs_fixed)
            targets_denorm = scaler.inverse_transform(batch_y_fixed)

            predictions.append(outputs_denorm)
            actuals.append(targets_denorm)

    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)

    mae = mean_absolute_error(actuals, predictions)

    mean_actual = np.mean(np.abs(actuals))

    # Compute Accuracy
    accuracy = max(0, 100 - (mae / mean_actual * 100))

    print(f"Test Accuracy: {accuracy:.2f}%")

evaluate_accuracy(model, train_dataloader, scaler)

Test Accuracy: 93.79%


training data, testinga data and evaluation data: all accuracy are similar hence model is perfect.